# Pandas로 MongoDB의 Data 처리하기

In [2]:
from pymongo import MongoClient

In [ ]:
# mongoDB에 접속할 client 생성
murl = ''

client = MongoClient(murl, 27017)

In [5]:
import mongo_conn

client = mongo_conn.get_client()

In [6]:
football_database = client.Football

In [7]:
match_collection = football_database.wc_match

In [8]:
match_collection.count_documents({})

64

## 국가별 득점 랭킹

### 국가별 득점 가져오기(PyMongo)

* **Data 구조 확인**
    - 경기 { 경기 일자, 홈팀 { 국가, 득점, 선수 { 선수기록들 } }, 원정팀 { 국가, 득점, 선수 { 선수기록들 } } }

In [9]:
list(match_collection.find({}))[0]

{'_id': ObjectId('5be05afeee0a5a6e9fe341e8'),
 'date': '14 JUN 2018 18:00',
 'home_team': {'score': 5,
  'players': [{'DC': 3606.0,
    'Total_PP': 0.52,
    'Medium_PA': 2.0,
    'TTP': 5808.0,
    'position': 'GK',
    'num': 1,
    'Short_PA': 1.0,
    'Total_PC': 11.0,
    'DCNIP': 1215.0,
    'Long_PC': 8.0,
    'name': 'Igor AKINFEEV',
    'Short_PC': 1.0,
    'SPRINT': 1.0,
    'Medium_PC': 2.0,
    'club': 'CSKA Moskva (RUS)',
    'DCIP': 1047.0,
    'Total_PA': 21.0,
    'Long_PA': 18.0,
    'TS': 21.46},
   {'DC': 10822.0,
    'Medium_PA': 16.0,
    'DCIP': 3452.0,
    'Short_PA': 4.0,
    'DCNIP': 4668.0,
    'Long_PC': 5.0,
    'name': 'MARIO FERNANDES',
    'Short_PC': 4.0,
    'Total_PP': 0.88,
    'SPRINT': 48.0,
    'club': 'CSKA Moskva (RUS)',
    'TA3RD': 0.12,
    'Total_PA': 25.0,
    'TPA': 0.01,
    'position': 'DF',
    'num': 2,
    'Medium_PC': 13.0,
    'Long_PA': 5.0,
    'TTP': 5808.0,
    'TOH': 0.34,
    'Total_PC': 22.0,
    'TS': 31.5},
   {'DC': 9987.0,

* 원하는 Data 가져오기
    - Aggregate(Group By)를 이용하여 국가별 홈경기 총 득점, 원정경기 총 득점 Load

In [11]:
# pymongo를 이용한 MongoDB 결과값은 Cursor의 형태
# Cursor는 일종의 휘발성 변수로 1회 호출 후 바로 Memory에서 삭제
# 반드시 List, DataFrame 등으로 전환해야 지속적으로 사용 가능

home_goals = list(match_collection.aggregate([
    {'$group': {
        '_id': '$home_team.country',
        'home_goal': {'$sum' : '$home_team.score'}
    }},
    {'$sort': {'home_goal': -1}}
]))

away_goals = list(match_collection.aggregate([
    {'$group': {
        '_id': '$away_team.country',
        'away_goal': {'$sum' : '$away_team.score'}
    }},
    {'$sort': {'away_goal': -1}}
]))

* MongoDB에는 Join이 없기 때문의 위의 두 결과(국가별 홈경기 총득점과 원정경기 총득점)의 합을 구하기가 어려움
* MongoDB에서 복잡하지 않게 가능한 수준의 Filtering, Aggregating 된 결과를 Pandas를 통해 분석 수행하는 것이 보다 용이

### 국가별 득점 합치기

In [12]:
import pandas as pd

home_goal_pd = pd.DataFrame(home_goals)
away_goal_pd = pd.DataFrame(away_goals)

In [13]:
home_goal_pd.head()

,_id,home_goal
0,Belgium,13
1,France,12
2,Russia,10
3,England,6
4,Brazil,6


In [14]:
away_goal_pd.head()

,_id,away_goal
0,Croatia,9
1,England,6
2,Argentina,5
3,Colombia,4
4,Tunisia,4


* Pandas에서 2개 이상의 DataFrame을 하나로 합치는 방법은 3가지가 존재
    1. Merge
    2. Join
    3. Concatenate

##### 1. Merge

In [19]:
# 기본적으로는 동일한 Column 명을 Key로 Inner Join 수행
pd.merge(home_goal_pd, away_goal_pd)

,_id,home_goal,away_goal
0,Belgium,13,3
1,France,12,2
2,Russia,10,1
3,England,6,6
4,Brazil,6,2
5,Uruguay,6,1
6,Croatia,5,9
7,Portugal,4,2
8,Korea Republic,3,0
9,Nigeria,3,0


In [20]:
# 경우에 따라 parameter를 통해 Join 방식(how), Join Key(on) 지정 가능
pd.merge(home_goal_pd, away_goal_pd, how='inner', on='_id')

,_id,home_goal,away_goal
0,Belgium,13,3
1,France,12,2
2,Russia,10,1
3,England,6,6
4,Brazil,6,2
5,Uruguay,6,1
6,Croatia,5,9
7,Portugal,4,2
8,Korea Republic,3,0
9,Nigeria,3,0


##### 2. Join

In [28]:
# Join을 이용하여 동일한 결과 가능
# Index를 기준으로 하는 것이 보편적
# 단, Join이 수행되는 두 DataFrame에 동일한 Column 명이 존재하는 경우 suffix 지정이 필수
home_goal_pd.set_index('_id').join(away_goal_pd.set_index('_id'), lsuffix='_home', rsuffix='_away')

,home_goal,away_goal
_id,,
Belgium,13,3
France,12,2
Russia,10,1
England,6,6
Brazil,6,2
Uruguay,6,1
Croatia,5,9
Portugal,4,2
Korea Republic,3,0


##### 3. Concatenate

In [29]:
# Concatenate는 2개 이상의 DataFrame을 그대로 이어 붙이는 형태 (일종의 UNION ALL)
pd.concat([home_goal_pd, away_goal_pd])

,_id,away_goal,home_goal
0,Belgium,NaN,13.0
1,France,NaN,12.0
2,Russia,NaN,10.0
3,England,NaN,6.0
4,Brazil,NaN,6.0
5,Uruguay,NaN,6.0
6,Croatia,NaN,5.0
7,Portugal,NaN,4.0
8,Korea Republic,NaN,3.0
9,Nigeria,NaN,3.0


### 홈경기 득점과 원정경기 득점을 병합하고 총 득점 계산

In [36]:
total_goals = pd.merge(home_goal_pd, away_goal_pd)
total_goals.head()

,_id,home_goal,away_goal
0,Belgium,13,3
1,France,12,2
2,Russia,10,1
3,England,6,6
4,Brazil,6,2


In [37]:
# total_goal = home_goal + away_gaol

total_goals['total_goals_1'] = total_goals.home_goal + total_goals.away_goal
total_goals.head()

,_id,home_goal,away_goal,total_goals_1
0,Belgium,13,3,16
1,France,12,2,14
2,Russia,10,1,11
3,England,6,6,12
4,Brazil,6,2,8


In [39]:
# Apply 함수와 Lambda 이용
# Apply 함수의 axis가 0인 경우 Column, 1인 경우 Row를 Iterate 하면서 Function을 수행

total_goals['total_goals_2'] = total_goals.apply(lambda x: x.home_goal + x.away_goal, axis=1)
total_goals.head()

,_id,home_goal,away_goal,total_goals_1,total_goals_2
0,Belgium,13,3,16,16
1,France,12,2,14,14
2,Russia,10,1,11,11
3,England,6,6,12,12
4,Brazil,6,2,8,8


### 득점 순으로 정렬하기

In [41]:
# sort_values 함수는 DataFrame을 특정 Column의 값을 기준으로 정렬
# ascending Option은 기본은 True(오름차순)이나 False로 지정시 내림차순으로 정렬

total_goals.sort_values('total_goals_2', ascending=False)

,_id,home_goal,away_goal,total_goals_1,total_goals_2
0,Belgium,13,3,16,16
6,Croatia,5,9,14,14
1,France,12,2,14,14
3,England,6,6,12,12
2,Russia,10,1,11,11
4,Brazil,6,2,8,8
5,Uruguay,6,1,7,7
10,Spain,3,4,7,7
12,Colombia,2,4,6,6
23,Argentina,1,5,6,6
